In [1]:
#inicjalizacja bibliotek
import pandas as pd
import numpy as np
from sympy import symbols, Eq, solve
import matplotlib.pyplot as plt
#metrics
from sklearn.metrics import mean_squared_error, r2_score

#GUI
import tkinter as tk
from tkinter import messagebox

import copy #creates deepcopy


from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.

#zapis do pdfu z listy figs
from matplotlib.backends.backend_pdf import PdfPages

In [144]:
class point:
    "Power_Element1 - power suply of Element1 target, Power_Element2 - power suply of Element2 target, Concentration_Element1 - concentrations in the middle of sample, total_thickness - total thickness of alloy"
    def __init__(self, Element1, Element2, Power_Element1, Power_Element2, Concentration_Element1, total_thickness, time):
        
        #data = {"Co":6.67*10**-6, "Ni":6.59*10**-6, "Au":10.21*10**-6, "Pt":9.09*10**-6, "Tb":1.93*10**-5} # dict with molecular volume of elements
        data = dict(pd.read_excel("molecular_database.xlsx"))
        
        self.Element1 = str(Element1)
        self.Element2 = str(Element2)
        self.Power_Element1 = float(Power_Element1)
        self.Power_Element2 = float(Power_Element2)
        self.Concentration_Element1 = float(Concentration_Element1)
        self.total_thickness = float(total_thickness)
        self.time = int(time)
        
        #inicjalizacja wartości stałych
        self.VElement1 = float(data[str(Element1)])
        self.VElement2 = float(data[str(Element2)])
        self.deposition_rate()
        
       
    def thickness(self):
        tElement11, tElement21 = symbols('tElement1, tElement2')
        eq1 = Eq(100*(tElement11/self.VElement1)/((tElement11/self.VElement1) + ((tElement21)/self.VElement2)),self.Concentration_Element1)
        eq2 =  Eq(tElement21 + tElement11,self.total_thickness)
        solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement11,tElement21)).items())
        self.tElement1  = solved['tElement1'] 
        self.tElement2 = solved['tElement2']
        return solved
    
    def deposition_rate(self):
        try:
            dr_solved = dict((key, value/self.time) for (key, value) in self.thickness().items())
            self.rate_Element1 = dr_solved['tElement1']
            self.rate_Element2 = dr_solved['tElement2']
            return dr_solved
        except ZeroDivisionError as Error:
            print("Time Error")
        

    @staticmethod
    def line(*samples):
        plt.clf()
        line_param={"Element":[],"Slope":[],"Intercept":[]}
        x = np.linspace(0,70,100)
        #firslty we do list comprehension, after that to flat array to 1D we use sum(list,[]) method, next we transform list to set to get only unique value, and as last we create empty dictionary by another list of comprehension
        element_dict = {i: [] for i in sum([[element.Element1, element.Element2] for element in samples],[])}
        power_dict = copy.deepcopy(element_dict)
        for sample in samples:
            element_dict[sample.Element1].append(sample.rate_Element1)
            element_dict[sample.Element2].append(sample.rate_Element2)
            power_dict[sample.Element1].append(sample.Power_Element1)
            power_dict[sample.Element2].append(sample.Power_Element2)

        for element in element_dict:
            fit = np.polyfit(x=power_dict[element], y=element_dict[element], deg=1)
            fit1D = np.poly1d(fit)

            line_param["Element"].append(element)
            line_param["Slope"].append(fit[0])
            line_param["Intercept"].append(fit[1])
            
            
            plt.scatter(x=power_dict[element], y=element_dict[element])
            plt.legend(sum([[key]*2 for key in element_dict.keys()],[])) # double keys in short a,b keys transform into a,a,b,b
            plt.plot(x,fit1D(x))
            #plt.rcParams["figure.figsize"] = (10,10)
            
        figs = list(map(plt.figure, plt.get_fignums()))
        return line_param, figs
    
    #@staticmethod
    #def draw(*samples):
            #plt.scatter(x=power_dict[element], y=element_dict[element])
            #plt.legend(sum([[key]*2 for key in element_dict.keys()],[])) # double keys in short a,b keys transform into a,a,b,b
            #plt.plot(x,fit1D(x))
    
    @staticmethod
    def save(*samples):
        data, fig = point.line(*samples)
        data = pd.DataFrame(data)
        # here if file doesnt exist. all calculated data will be saved into .xlsx file
        try:
            old = pd.read_excel("sputtering_parameters.xlsx",index_col=[0])
            for element in data["Element"]:
                # here we check if old data contains information about each element calculated today if not it will add new row in data, if contains it will update information
                try:
                    test = old.loc[old["Element"]==element,"Slope"].item() # chceck procedure if fail will skip to excpet procedure
                    old.loc[old["Element"]==element,"Slope"]=data.loc[data["Element"]==element,"Slope"].item()
                    old.loc[old["Element"]==element,"Intercept"]=data.loc[data["Element"]==element,"Slope"].item()
                except ValueError:
                    old = pd.concat((old,data.loc[data["Element"]==element]),axis=0).reset_index(drop=True)
            old.to_excel("sputtering_parameters.xlsx")        
        except FileNotFoundError:
            pd.DataFrame(data).to_excel("sputtering_parameters.xlsx")
            

# GUI

In [146]:
%%capture 
class App_window():
    
    def __init__(self, point):
        #data = dict(pd.read_excel("molecular_database.xlsx"))
        self.point = point
        
        #root initialization
        self.root = tk.Tk()
        self.root.title("Sputtering Calibration by Łukasz Frąckowiak")
        self.root.geometry("700x600")
        self.root.resizable(False, False)
        self.logic = False
        self.logic2 = False
        #cordinations initialization
        self.x, self.y = 0.2, 0.1 
        #self.check_var = True
        #self.entry_list = []
        #elements of windows first frame
        self.start_frame()
        #self.Add()
        self.create_field()
        self.root.mainloop()
        
        
     
    def start_frame(self):
        self.text = tk.Label(text="Please enter: \nname of 1st element and 2nd element,\n Power of element_1 [W], Power of element_2 [W], \nconcentration of element_1 in the center of sample [at.%], \ntotal alloy thickness [A], sputtering time [s] \n e.g. Co, Au, 10, 20, 12, 80, 100", font=("", 10))
        self.text.place(relx=0, rely=0,)
        
        self.button_exit = tk.Button(text="EXIT", command = self._exit)
        self.button_exit.place(relx=0.1, rely=0.9, anchor='sw')
        
        self.button_calculate = tk.Button(text="Calculate", command = self.calculate)
        self.button_calculate.place(relx=0.6, rely=0.05)
        """
    def Add(self):
        if self.logic:
                self.entry_list.append(self.entry.get())
        else:
            self.logic=True
        #self.entry_list.append(self.entry.get()) if self.logic else self.logic=True  #save values from entry into entry_list
        self.button_new = tk.Button(text="Add new point", command= self.create_field)
        self.button_new.place(relx = self.x+0.07, rely= self.y+0.05)
        """
        
    def create_field(self):
        
        self.entry1 = tk.Entry()
        self.entry1.place(relx= self.x-0.15, rely= self.y+0.105)
        
        self.entry2 = tk.Entry()
        self.entry2.place(relx= self.x-0.15, rely= self.y+0.155)
        
        self.entry3 = tk.Entry()
        self.entry3.place(relx= self.x-0.15, rely= self.y+0.205)
        
        self.entry4 = tk.Entry()
        self.entry4.place(relx= self.x-0.15, rely= self.y+0.255)
        
        self.entry5 = tk.Entry()
        self.entry5.place(relx= self.x-0.15, rely= self.y+0.305)
        
        self.entry6 = tk.Entry()
        self.entry6.place(relx= self.x-0.15, rely= self.y+0.355)
        
        self.entry7 = tk.Entry()
        self.entry7.place(relx= self.x-0.15, rely= self.y+0.405)
        
        self.entry8 = tk.Entry()
        self.entry8.place(relx= self.x-0.15, rely= self.y+0.455)
        
        self.entry9 = tk.Entry()
        self.entry9.place(relx= self.x-0.15, rely= self.y+0.505)
        
        self.entry10 = tk.Entry()
        self.entry10.place(relx= self.x-0.15, rely= self.y+0.555)
    """ 
        #self.button1.place(relx= self.x-0.1, rely= self.y+0.05)
        if self.logic2:
                self.entry_list.append(self.entry.get())
        else:
            self.logic2=True
        self.entry = tk.Entry()
        self.entry.place(relx= self.x-0.15, rely= self.y+0.055)
        
        self.y += 0.05
        self.y += 0.05
        #self.button_check = tk.Button(text="✅", height= 1,command=self.check)
        #self.button_check.place(relx= self.x_but2, rely=self.y_ent)
       """
    def calculate(self):
        
        self.button_save = tk.Button(text="SAVE", command = self.save)
        self.button_save.place(relx=0.8, rely=0.05)
        
        self.button_export = tk.Button(text="Export plot", command = self.export)
        self.button_export.place(relx=0.7, rely=0.05)
        
        """
        self.entry_list.append(self.entry.get()) #save values from entry into entry_list
        self.text2 = tk.Label(text=f"{self.entry_list}", font=("", 10))
        self.text2.place(relx=0.5, rely=0.5)
        self.entry_list.pop() #prevent addint new element to entry_list by multiple clicking of calculate button
        """
        """
        def error_entry(point)
            if len(point_list)<2:
                messagebox.showerror("warning", "Number of points is not sufficient to give correct results")
            for point in point_list:
                if len(point) != 7:
                    messagebox.showerror("error", "Please correct point entry")
        """
        def transform(entryWidget):
            lista=[]
            for entry in entryWidget.replace(" ","").split(sep=","):
                try:
                    lista.append(float(entry))
                except ValueError:
                    lista.append(str(entry))
            return lista        
            
        point_list = []
        entry_list = map(transform,[self.entry1.get(),self.entry2.get(),self.entry3.get(),self.entry4.get(),self.entry5.get(),self.entry6.get(),self.entry7.get(),self.entry8.get(),self.entry9.get(),self.entry10.get()])
        
        for entry in entry_list:
            try:
                 point_list.append(point(*entry))
            except KeyError:
                messagebox.showerror("Error 1", "At least one of element does not appear in database.\nPlease check spelling or fill molecular_database.xlsx entry.")
            except TypeError:
                if len(entry)!=1:
                    messagebox.showerror("Error 2", "You input wrong number of parameters.\nPlease correct entry values")
            except ValueError:
                messagebox.showerror("Error 3", "Please correct entry values")
        self.point_list = point_list        
        data_points, self.fig = point.line(*point_list)
        self.lists_of_points = pd.DataFrame(data_points)    
        self.text2 = tk.Label(text=f"{self.lists_of_points}", font=("", 10))
        self.text2.place(relx=0.5, rely=0.2)
        

        
        canvas = FigureCanvasTkAgg(self.fig[0])  # A tk.DrawingArea.
        canvas.draw()
        canvas.get_tk_widget().place(relx=0.25, rely=0.4)
        canvas.get_tk_widget().configure(width=500,height=300)
        

    def save(self):
        point.save(*self.point_list)
        
    def _exit(self):
        self.root.quit()  # stops mainloop
        self.root.destroy()  # this is necessary on Windows to preven

    def export(self):
        
        pdf = PdfPages('plot_image.pdf')
        pdf.savefig(self.fig[0])
        pdf.close()

            

App_window(point)